In [1]:
import sys
sys.path.append("../generate_signal")
sys.path.append("../framework")

from account_class import account
from system_function import get_price
from system_function import get_today_open_price
from system_function import get_total_value
from system_function import get_total_value_in_a_monnt
from system_function import get_total_value_every_day
from system_function import stop_lost
from system_function import order_buy
from system_function import order_sell
from system_function import borrow_stock
from system_function import return_stock

In [2]:
today="2023-11-03"

In [3]:
import pandas as pd
Close=pd.read_csv("../期货数据/"+today+"/Afuturesclose1m.csv",index_col=0)
Open=pd.read_csv("../期货数据/"+today+"/Afuturesopen1m.csv",index_col=0)
High=pd.read_csv("../期货数据/"+today+"/Afutureshigh1m.csv",index_col=0)
Low=pd.read_csv("../期货数据/"+today+"/Afutureslow1m.csv",index_col=0)
Volume=pd.read_csv("../期货数据/"+today+"/Afuturesvolume1m.csv",index_col=0)
Money=pd.read_csv("../期货数据/"+today+"/Afuturesmoney1m.csv",index_col=0)

In [4]:
Open=Open.dropna(axis=1,how='all')
High=High.dropna(axis=1,how='all')
Low=Low.dropna(axis=1,how='all')
Close=Close.dropna(axis=1,how='all')
Volume=Volume.dropna(axis=1,how='all')
Money=Money.dropna(axis=1,how='all')

In [5]:
import numpy as np

# 协整检验

In [6]:
#statsmodels.tsa.stattools.adfuller

import statsmodels.tsa.stattools as ts
# 创建一个函数用来协整检验
def cointegration_check(series01, series02):

    urt_rb2001 = ts.adfuller(np.array(series01), 1)[1]
    urt_rb2005 = ts.adfuller(np.array(series02), 1)[1]
    #print(urt_rb2001)
    #print(urt_rb2005)
    # 同时平稳或不平稳则差分再次检验
    if (urt_rb2001 > 0.1 and urt_rb2005 > 0.1) or (urt_rb2001 < 0.1 and urt_rb2005 < 0.1):
        urt_diff_rb2001 = ts.adfuller(np.diff(np.array(series01)), 1)[1]
        urt_diff_rb2005 = ts.adfuller(np.diff(np.array(series02), 1))[1]

        # 同时差分平稳进行OLS回归的残差平稳检验
        if urt_diff_rb2001 < 0.1 and urt_diff_rb2005 < 0.1:
            matrix = np.vstack([series02, np.ones(len(series02))]).T
            beta, c = np.linalg.lstsq(matrix, series01)[0]
            resid = series01 - beta * series02 - c
            if ts.adfuller(np.array(resid), 1)[1] > 0.1:
                result = 0.0
            else:
                result = 1.0
            return beta, c, resid, result
        else:
            result = 0.0
            return 0.0, 0.0, 0.0, result

    else:
        result = 0.0
        return 0.0, 0.0, 0.0, result

# OU过程的参数估计

In [7]:
from dataclasses import dataclass

@dataclass
class OUParams:
    alpha: float  # mean reversion parameter
    gamma: float  # asymptotic mean
    beta: float  # Brownian motion scale (standard deviation)

In [8]:
from sklearn.linear_model import LinearRegression

def estimate_OU_params(X_t: np.ndarray) -> OUParams:
    """
    Estimate OU params from OLS regression.
    - X_t is a 1D array.
    Returns instance of OUParams.
    """
    y = np.diff(X_t)
    X = X_t[:-1].reshape(-1, 1)
    reg = LinearRegression(fit_intercept=True)
    reg.fit(X, y)
    # regression coeficient and constant
    alpha = -reg.coef_[0]
    gamma = reg.intercept_ / alpha
    # residuals and their standard deviation
    y_hat = reg.predict(X)
    beta = np.std(y - y_hat)
    return OUParams(alpha, gamma, beta)

# 生成交易信号上下界

In [9]:
def generate_up_down(d,l,i,k):
    futures1=sta_pair[i][0]
    futures2=sta_pair[i][1]
    today_index=Close.index[d]
    #print(today_index)
    yesterday_index=Close.index[d-1]
    #print(yesterday_index)
    l_day_ago_index=Close.index[d-l]
    windows_close=Close.loc[l_day_ago_index:yesterday_index,[futures1,futures2]]

    diff_price=pd.DataFrame(abs(windows_close[futures1]-windows_close[futures2]))
    diff_price.columns=[sta_pair[i][1]+" and "+sta_pair[i][0]+"价差"]
    diff_price=diff_price.dropna()
    #print(diff_price)
    OU_params_hat = estimate_OU_params(np.array(diff_price[sta_pair[i][1]+" and "+sta_pair[i][0]+"价差"].dropna()))
    mean_diff=np.mean(np.array(diff_price[sta_pair[i][1]+" and "+sta_pair[i][0]+"价差"].dropna()))
    std_diff=math.sqrt(np.var(np.array(diff_price[sta_pair[i][1]+" and "+sta_pair[i][0]+"价差"].dropna()))/(2*OU_params_hat.alpha))

    up_open=mean_diff+k*std_diff
    down_open=mean_diff-k*std_diff
    return mean_diff,up_open,down_open

# 账号

In [10]:
cyf=account("dont_care","dont_care",10000000,Close)

# 计算昨天结算价

In [11]:
import numpy as np
def get_futures_ysterday_jie_suan_price(futures_code,time,Close,Volume):
    bussiness=["IF9999.CCFX","TF9999.CCFX","T9999.CCFX","IC9999.CCFX","IH9999.CCFX","TS9999.CCFX","IM9999.CCFX","TL9999.CCFX"]
    #按照黄金开盘收盘时间生成的数据，一天555条数据
    if futures_code not in bussiness:
        #对于商品期货
        if "09:01:00"<=time.split(" ")[1]<="23:59:00":#在同一天
            yesterday_close_index=np.where(Close.index==time.split(" ")[0]+" 02:30:00")[0][0]
            today_open_index=np.where(Close.index==time.split(" ")[0]+" 09:01:00")[0][0]
        
            yesterday_open_index=today_open_index-555
        else:#不在同一天   00:00:00-02:30:00
            today_0_index=np.where(Close.index==time.split(" ")[0]+" 00:00:00")[0][0]
            yesterday_close_index=today_0_index-405
           
            
            
            
            yesterday_open_index=today_0_index-555-404
            
            
        
        yesterday_close_d=Close.index[yesterday_close_index]
        yesterday_open_d=Close.index[yesterday_open_index]
        #print(Close.index[yesterday_close_index])
        #print(Close.index[yesterday_open_index])
        
        yesterday_jie_suan_price=sum(np.array(Volume.loc[yesterday_open_d:yesterday_close_d,[futures_code]].dropna())*\
        np.array(Close.loc[yesterday_open_d:yesterday_close_d,[futures_code]].dropna()))\
        /sum(np.array(Volume.loc[yesterday_open_d:yesterday_close_d,[futures_code]].dropna()))
        
        return float(yesterday_jie_suan_price)
    
   

# 买 卖

In [12]:
#买入
def order_buy_futures(account,futures_code,time,amount,huadian,Close,Volume):
    fee_dict={'AG': 0.50/100/100,'AL': 3/100/100,'AU': 10/100/100,'BU': 1.00/100/100,'CU': 0.50/100/100,'FU': 0.10/100/100,
              'HC': 1.00/100/100,'NI': 3/100/100,'PB': 0.40/100/100,'RB': 1.00/100/100,'RU': 3.00/100/100,'SN': 3/100/100,
              'WR': 0.40/100/100,'ZN': 3/100/100,'SP': 0.50/100/100,'SS': 2/100/100,'A': 2/100/100,'B': 1/100/100,
              'BB': 1.00/100/100,'C': 2/100/100,'CS': 1.5/100/100,'FB': 1.00/100/100,'I': 1.00/100/100,'J': 1.00/100/100,
              'JD': 1.50/100/100,'JM': 1.00/100/100,'L': 1/100/100,'M': 1.5/100/100,'P': 2.5/100/100,'PP': 1/100/100,
              'V': 1/100/100,'Y': 2.5/100/100,'EG': 3/100/100,'RR': 1/100/100,'EB': 3/100/100,'PG': 6/100/100,
              'LH': 1.00/100/100,'AP': 5/100/100,'CF': 4.3/100/100,'CY': 4/100/100,'FG': 6/100/100,'JR': 3/100/100,
              'LR': 3/100/100,'MA': 1.00/100/100,'OI': 2/100/100,'PM': 30/100/100,'RI': 2.5/100/100,'RM': 1.5/100/100,
              'RS': 2/100/100,'SF': 3/100/100,'SM': 3/100/100,'SR': 3/100/100,'TA': 3/100/100,'WH': 30/100/100,
              'ZC': 150/100/100,'UR': 1.00/100/100,'PF': 3/100/100,'PK': 4/100/100,'SA': 3.5/100/100,'IC': 0.23/100/100,
              'IF': 0.23/100/100,'IH': 0.23/100/100,'IM': 0.23/100/100,'T': 3/100/100,'TF': 3/100/100,'TL': 3/100/100,
              'TS': 3/100/100,'BC': 0.10/100/100,'LU': 0.10/100/100,'NR': 0.20/100/100,'SC': 20/100/100}
    #涨跌幅限制，就不考虑第一涨停，第二天又涨停，第三天又涨停的限制了
    earn_lost_stop_normal=\
{'AG': 9.0/100,
 'AL': 7.0/100,
 'AU': 7.0/100,
 'BU': 10.0/100,
 'CU': 7.0/100,
 'FU': 13.0/100,
 'HC': 11.0/100,
 'NI': 10.0/100,
 'PB': 7.0/100,
 'RB': 11.0/100,
 'RU': 8.0/100,
 'SN': 12.0/100,
 'WR': 11.0/100,
 'ZN': 7.0/100,
 'SP': 8.0/100,
 'SS': 8.0/100,
 'A': 6.0/100,
 'B': 6.0/100,
 'BB': 5.0/100,
 'C': 6.0/100,
 'CS': 5.0/100,
 'FB': 5.0/100,
 'I': 11.0/100,
 'J': 15.0/100,
 'JD': 6.0/100,
 'JM': 15.0/100,
 'L': 6.0/100,
 'M': 6.0/100,
 'P': 7.0/100,
 'PP': 6.0/100,
 'V': 6.0/100,
 'Y': 6.0/100,
 'EG': 7.0/100,
 'RR': 5.0/100,
 'EB': 7.0/100,
 'PG': 7.0/100,
 'LH': 8.0/100,
 'AP': 9.0/100,
 'CF': 6.0/100,
 'CY': 6.0/100,
 'FG': 8.0/100,
 'JR': 7.0/100,
 'LR': 7.0/100,
 'MA': 7.0/100,
 'OI': 8.0/100,
 'PM': 7.0/100,
 'RI': 7.0/100,
 'RM': 8.0/100,
 'RS': 10.0/100,
 'SF': 10.0/100,
 'SM': 10.0/100,
 'SR': 8.0/100,
 'TA': 6.0/100,
 'WH': 7.0/100,
 'ZC': 10.0/100,
 'UR': 7.0/100,
 'PF': 7.0/100,
 'PK': 7.0/100,
 'SA': 8.0/100,
 'IC': 10.0/100,
 'IF': 10.0/100,
 'IH': 10.0/100,
 'IM': 10.0/100,
 'T': 2.0/100,
 'TF': 1.2/100,
 'TL': 10000000.0/100,
 'TS': 0.5/100,
 'BC': 7.0/100,
 'LU': 10.0/100,
 'NR': 8.0/100,
 'SC': 13.0/100}
    #只单边收费
    only_sell=['AU', 'FU', 'PB', 'RU', 'WR', 'ZN', 'SP', 'SS', 'CF', 'CY', 'SF', 'SM', 'SR', 'TA', 'T', 'TF', 'TL', 'TS', 'BC', 'NR', 'SC']
    futures_type=futures_code.split(".")[0].replace("9999","")
    if futures_type in only_sell:
        fee=0
    else:
        if futures_type in fee_dict.keys():
            fee=fee_dict[futures_type]
        else:
            fee=0
    
 
    
    
    if futures_type in earn_lost_stop_normal and\
    (get_price(futures_code,time,Close)-get_futures_ysterday_jie_suan_price(futures_code,time,Close,Volume))/\
    get_futures_ysterday_jie_suan_price(futures_code,time,Close,Volume)>=earn_lost_stop_normal[futures_type]:
        #涨停无法买入可卖出
        return "涨停"
    else:
        single_price=get_price(futures_code,time,Close)+huadian
    
        #print(single_price)
    
        total_price0=single_price*amount
        #print(total_price0)
        total_price=total_price0*(1+fee)
        #print(total_price)
    
        #print(account.get_coin())
        #print(account.get_coin()>=total_price)
        if account.get_coin()>=total_price:#这个if条件语句有个意想不到的收获，解决了股票数据nan的问题，反正都是false 不会交易
            account.update_coin(account.get_coin()-total_price)
            account.update_stock(futures_code,account.stock[futures_code]+amount)
            return "Aoocunt coin: "+str(account.get_coin())+"\n"+futures_code+":"+str(account.stock[futures_code])+"\n"\
                    +"期货"+futures_code+"的数量为: "+str(account.stock[futures_code])+'\n'
        else:
            return "no money don't touch! \n Aoocunt coin:"+str(account.get_coin())
        


In [13]:
#卖出
def order_sell_futures(account,futures_code,time,sell_amount,huadian,Close,Volume):#fee 卖出手续费 tax印花税 amount:卖多少股
    fee_dict={'AG': 0.50/100/100,'AL': 3/100/100,'AU': 10/100/100,'BU': 1.00/100/100,'CU': 0.50/100/100,'FU': 0.10/100/100,
              'HC': 1.00/100/100,'NI': 3/100/100,'PB': 0.40/100/100,'RB': 1.00/100/100,'RU': 3.00/100/100,'SN': 3/100/100,
              'WR': 0.40/100/100,'ZN': 3/100/100,'SP': 0.50/100/100,'SS': 2/100/100,'A': 2/100/100,'B': 1/100/100,
              'BB': 1.00/100/100,'C': 2/100/100,'CS': 1.5/100/100,'FB': 1.00/100/100,'I': 1.00/100/100,'J': 1.00/100/100,
              'JD': 1.50/100/100,'JM': 1.00/100/100,'L': 1/100/100,'M': 1.5/100/100,'P': 2.5/100/100,'PP': 1/100/100,
              'V': 1/100/100,'Y': 2.5/100/100,'EG': 3/100/100,'RR': 1/100/100,'EB': 3/100/100,'PG': 6/100/100,
              'LH': 1.00/100/100,'AP': 5/100/100,'CF': 4.3/100/100,'CY': 4/100/100,'FG': 6/100/100,'JR': 3/100/100,
              'LR': 3/100/100,'MA': 1.00/100/100,'OI': 2/100/100,'PM': 30/100/100,'RI': 2.5/100/100,'RM': 1.5/100/100,
              'RS': 2/100/100,'SF': 3/100/100,'SM': 3/100/100,'SR': 3/100/100,'TA': 3/100/100,'WH': 30/100/100,
              'ZC': 150/100/100,'UR': 1.00/100/100,'PF': 3/100/100,'PK': 4/100/100,'SA': 3.5/100/100,'IC': 0.23/100/100,
              'IF': 0.23/100/100,'IH': 0.23/100/100,'IM': 0.23/100/100,'T': 3/100/100,'TF': 3/100/100,'TL': 3/100/100,
              'TS': 3/100/100,'BC': 0.10/100/100,'LU': 0.10/100/100,'NR': 0.20/100/100,'SC': 20/100/100}
        
    #涨跌幅限制，就不考虑第一涨停，第二天又涨停，第三天又涨停的限制了
    earn_lost_stop_normal=\
{'AG': 9.0/100,
 'AL': 7.0/100,
 'AU': 7.0/100,
 'BU': 10.0/100,
 'CU': 7.0/100,
 'FU': 13.0/100,
 'HC': 11.0/100,
 'NI': 10.0/100,
 'PB': 7.0/100,
 'RB': 11.0/100,
 'RU': 8.0/100,
 'SN': 12.0/100,
 'WR': 11.0/100,
 'ZN': 7.0/100,
 'SP': 8.0/100,
 'SS': 8.0/100,
 'A': 6.0/100,
 'B': 6.0/100,
 'BB': 5.0/100,
 'C': 6.0/100,
 'CS': 5.0/100,
 'FB': 5.0/100,
 'I': 11.0/100,
 'J': 15.0/100,
 'JD': 6.0/100,
 'JM': 15.0/100,
 'L': 6.0/100,
 'M': 6.0/100,
 'P': 7.0/100,
 'PP': 6.0/100,
 'V': 6.0/100,
 'Y': 6.0/100,
 'EG': 7.0/100,
 'RR': 5.0/100,
 'EB': 7.0/100,
 'PG': 7.0/100,
 'LH': 8.0/100,
 'AP': 9.0/100,
 'CF': 6.0/100,
 'CY': 6.0/100,
 'FG': 8.0/100,
 'JR': 7.0/100,
 'LR': 7.0/100,
 'MA': 7.0/100,
 'OI': 8.0/100,
 'PM': 7.0/100,
 'RI': 7.0/100,
 'RM': 8.0/100,
 'RS': 10.0/100,
 'SF': 10.0/100,
 'SM': 10.0/100,
 'SR': 8.0/100,
 'TA': 6.0/100,
 'WH': 7.0/100,
 'ZC': 10.0/100,
 'UR': 7.0/100,
 'PF': 7.0/100,
 'PK': 7.0/100,
 'SA': 8.0/100,
 'IC': 10.0/100,
 'IF': 10.0/100,
 'IH': 10.0/100,
 'IM': 10.0/100,
 'T': 2.0/100,
 'TF': 1.2/100,
 'TL': 10000000.0/100,
 'TS': 0.5/100,
 'BC': 7.0/100,
 'LU': 10.0/100,
 'NR': 8.0/100,
 'SC': 13.0/100}
    futures_type=futures_code.split(".")[0].replace("9999","")
    if futures_type in fee_dict.keys():
        fee=fee_dict[futures_type]
    else:
        fee=0
    
    
    

    
    if futures_type in earn_lost_stop_normal and\
    (get_price(futures_code,time,Close)-get_futures_ysterday_jie_suan_price(futures_code,time,Close,Volume))\
    /get_futures_ysterday_jie_suan_price(futures_code,time,Close,Volume)<=-earn_lost_stop_normal[futures_type]:
        #跌停无法卖出但能买入
        return "跌停"
    else:
        single_price=get_price(futures_code,time,Close)-huadian
        print(futures_code+"卖之前股数： "+str(account.stock[futures_code]))
        #if account.stock[stock_code]>=sell_amount:期货在数量为0时依然可以卖出，但是要保证金，这里没写保证金
        account.update_stock(futures_code,account.stock[futures_code]-sell_amount)
        total_price=single_price*sell_amount
        account.update_coin(account.get_coin()+total_price*(1-fee))
        return "Aoocunt coin: "+str(account.get_coin())+"\n"+futures_code+":"+str(account.stock[futures_code])+"\n"
        #else:
            #return "no stock to sell! \n "+stock_code+":"+str(account.stock[stock_code])

# 平仓

In [14]:
def end_transation_one_futures(account,futures_code,amount1,d,Close):#平一只期货的仓

    if amount1>=0:
        order_sell_futures(account,futures_code,d,amount1,0.05,Close,Volume)
    if amount1<0:
        order_buy_futures(account,futures_code,d,abs(amount1),0.05,Close,Volume)

In [15]:
def end_transation(account,sta_pair_list,d,Close):#["a","b",[0,0]]
    end_transation_one_futures(account,sta_pair_list[0],sta_pair_list[2][0],d,Close)
    end_transation_one_futures(account,sta_pair_list[1],sta_pair_list[2][1],d,Close)
        

# 开仓时计算账户的资产

In [16]:
def get_total_value_of_futures_in_a_moment(cyf,d,Close):
    fee_dict={'AG': 0.50/100/100,'AL': 3/100/100,'AU': 10/100/100,'BU': 1.00/100/100,'CU': 0.50/100/100,'FU': 0.10/100/100,
              'HC': 1.00/100/100,'NI': 3/100/100,'PB': 0.40/100/100,'RB': 1.00/100/100,'RU': 3.00/100/100,'SN': 3/100/100,
              'WR': 0.40/100/100,'ZN': 3/100/100,'SP': 0.50/100/100,'SS': 2/100/100,'A': 2/100/100,'B': 1/100/100,
              'BB': 1.00/100/100,'C': 2/100/100,'CS': 1.5/100/100,'FB': 1.00/100/100,'I': 1.00/100/100,'J': 1.00/100/100,
              'JD': 1.50/100/100,'JM': 1.00/100/100,'L': 1/100/100,'M': 1.5/100/100,'P': 2.5/100/100,'PP': 1/100/100,
              'V': 1/100/100,'Y': 2.5/100/100,'EG': 3/100/100,'RR': 1/100/100,'EB': 3/100/100,'PG': 6/100/100,
              'LH': 1.00/100/100,'AP': 5/100/100,'CF': 4.3/100/100,'CY': 4/100/100,'FG': 6/100/100,'JR': 3/100/100,
              'LR': 3/100/100,'MA': 1.00/100/100,'OI': 2/100/100,'PM': 30/100/100,'RI': 2.5/100/100,'RM': 1.5/100/100,
              'RS': 2/100/100,'SF': 3/100/100,'SM': 3/100/100,'SR': 3/100/100,'TA': 3/100/100,'WH': 30/100/100,
              'ZC': 150/100/100,'UR': 1.00/100/100,'PF': 3/100/100,'PK': 4/100/100,'SA': 3.5/100/100,'IC': 0.23/100/100,
              'IF': 0.23/100/100,'IH': 0.23/100/100,'IM': 0.23/100/100,'T': 3/100/100,'TF': 3/100/100,'TL': 3/100/100,
              'TS': 3/100/100,'BC': 0.10/100/100,'LU': 0.10/100/100,'NR': 0.20/100/100,'SC': 20/100/100}
    
    
    
    end_coin=cyf.get_coin() #当前时刻账户上的资金   
    stock_value=0
    universe=list(Close.columns)[0:len(Close.columns)]
    
  
            
    for jjj in range(len(universe)):#对于每一只股票
        if cyf.get_stock(universe[jjj])!=0:#股票数量不为0
            futures_type=universe[jjj].split(".")[0].replace("9999","")
    
            fee=fee_dict[futures_type]
        
            stock_value=stock_value+get_price(universe[jjj],d,Close)\
            *cyf.get_stock(universe[jjj])*(1-fee)#当分钟股票开盘价*股票数量 
    total_value=end_coin+stock_value

    
    return total_value


# 测试一下资产的计算

In [17]:
import numpy as np
cyf=account("dont_care","dont_care",10000000,Close)

order_sell_futures(cyf,'A9999.XDCE','2021-11-03 09:08:00',10000,0.05,Close,Volume)
order_buy_futures(cyf,'A9999.XDCE','2021-11-03 09:08:00',1000,0.05,Close,Volume)

A9999.XDCE卖之前股数： 0


'Aoocunt coin: 66451647.28999999\nA9999.XDCE:-9000\n期货A9999.XDCE的数量为: -9000\n'

In [18]:
get_total_value_of_futures_in_a_moment(cyf,'2021-11-03 09:08:00',Close)

9996940.489999987

In [19]:
end_transation_one_futures(cyf,'A9999.XDCE',-100,'2021-11-03 09:08:00',Close)

In [20]:
cyf.get_coin()

65824116.80899999

In [21]:
cyf.stock

{'A9999.XDCE': -8900,
 'AG9999.XSGE': 0,
 'AL9999.XSGE': 0,
 'AO9999.XSGE': 0,
 'AP9999.XZCE': 0,
 'AU9999.XSGE': 0,
 'B9999.XDCE': 0,
 'BB9999.XDCE': 0,
 'BC9999.XINE': 0,
 'BR9999.XSGE': 0,
 'BU9999.XSGE': 0,
 'C9999.XDCE': 0,
 'CF9999.XZCE': 0,
 'CJ9999.XZCE': 0,
 'CS9999.XDCE': 0,
 'CU9999.XSGE': 0,
 'CY9999.XZCE': 0,
 'EB9999.XDCE': 0,
 'EC9999.XINE': 0,
 'EG9999.XDCE': 0,
 'FB9999.XDCE': 0,
 'FG9999.XZCE': 0,
 'FU9999.XSGE': 0,
 'HC9999.XSGE': 0,
 'I9999.XDCE': 0,
 'J9999.XDCE': 0,
 'JD9999.XDCE': 0,
 'JM9999.XDCE': 0,
 'JR9999.XZCE': 0,
 'L9999.XDCE': 0,
 'LC9999.GFEX': 0,
 'LH9999.XDCE': 0,
 'LR9999.XZCE': 0,
 'LU9999.XINE': 0,
 'M9999.XDCE': 0,
 'MA9999.XZCE': 0,
 'NI9999.XSGE': 0,
 'NR9999.XINE': 0,
 'OI9999.XZCE': 0,
 'P9999.XDCE': 0,
 'PB9999.XSGE': 0,
 'PF9999.XZCE': 0,
 'PG9999.XDCE': 0,
 'PK9999.XZCE': 0,
 'PM9999.XZCE': 0,
 'PP9999.XDCE': 0,
 'PX9999.XZCE': 0,
 'RB9999.XSGE': 0,
 'RI9999.XZCE': 0,
 'RM9999.XZCE': 0,
 'RR9999.XDCE': 0,
 'RS9999.XZCE': 0,
 'RU9999.XSGE': 

# 总的交易算法

In [22]:
def find_pair_index(pair_list,a,b):
    for i in range(len(pair_list)):
        if set([a,b]) in pair_list[i]:
            try:
                index=pair_list.index([set([a,b]), 0])
            except:
                index=pair_list.index([set([a,b]), 1])
            
    return index

In [23]:
futures_name={
"IF9999.CCFX":"沪深300合约",
"TF9999.CCFX":"5年期国债合约",
"T9999.CCFX":"10年期国债合约",
"IC9999.CCFX":"中证500合约",
"IH9999.CCFX":"上证50合约",
"TS9999.CCFX":"2年期国债合约",
"IM9999.CCFX":"中证1000合约",
"TL9999.CCFX":"30年期国债合约",
"SI9999.GFEX":"工业硅合约",
"LC9999.GFEX":"碳酸锂合约",
"A9999.XDCE":"豆一合约",
"B9999.XDCE":"豆二合约",
"M9999.XDCE":"豆粕合约",
"C9999.XDCE":"玉米合约",
"Y9999.XDCE":"豆油合约",
"L9999.XDCE":"聚乙烯合约",
"P9999.XDCE":"棕榈油合约",
"V9999.XDCE":"聚氯乙烯合约",
"J9999.XDCE":"焦炭合约",
"JM9999.XDCE":"焦煤合约",
"I9999.XDCE":"铁矿石合约",
"JD9999.XDCE":"鸡蛋合约",
"BB9999.XDCE":"胶合板合约",
"FB9999.XDCE":"纤维板合约",
"PP9999.XDCE":"聚丙烯合约",
"CS9999.XDCE":"玉米淀粉合约",
"EG9999.XDCE":"乙二醇合约",
"RR9999.XDCE":"粳米合约",
"EB9999.XDCE":"苯乙烯合约",
"PG9999.XDCE":"液化石油气合约	",
"LH9999.XDCE":"生猪合约",
"SC9999.XINE":"原油合约",
"NR9999.XINE":"20号胶合约",
"LU9999.XINE":"低硫燃料油合约",
"BC9999.XINE":"阴极铜合约",
"EC9999.XINE":"集运指数（欧线）合约",
"AL9999.XSGE":"铝合约",
"RU9999.XSGE":"天然橡胶合约",
"FU9999.XSGE":"燃料油合约",
"CU9999.XSGE":"铜合约",
"ZN9999.XSGE":"锌合约",
"AU9999.XSGE":"黄金合约",
"WR9999.XSGE":"线材合约",
"RB9999.XSGE":"螺纹钢合约",
"PB9999.XSGE":"铅合约",
"AG9999.XSGE":"白银合约",
"BU9999.XSGE":"石油沥青合约",
"HC9999.XSGE":"热轧卷板合约",
"SN9999.XSGE":"锡合约",
"NI9999.XSGE":"镍合约",
"SP9999.XSGE":"纸浆合约",
"SS9999.XSGE":"不锈钢合约",
"AO9999.XSGE":"氧化铝合约",
"BR9999.XSGE":"丁二烯橡胶合约",
"WT9999.XZCE":"硬白小麦合约",
"WS9999.XZCE":"强麦合约",
"GN9999.XZCE":"绿豆合约",
"CF9999.XZCE":"棉花合约",
"SR9999.XZCE":"白糖合约",
"TA9999.XZCE":"PTA合约",
"RO9999.XZCE":"菜籽油合约",
"ER9999.XZCE":"早籼稻合约",
"ME9999.XZCE":"甲醇合约",
"PM9999.XZCE":"普麦合约",
"OI9999.XZCE":"菜籽油合约",
"WH9999.XZCE":"强麦合约",
"RI9999.XZCE":"早籼稻合约",
"FG9999.XZCE":"玻璃合约",
"RM9999.XZCE":"菜籽粕合约",
"RS9999.XZCE":"油菜籽合约",
"TC9999.XZCE":"动力煤合约",
"JR9999.XZCE":"粳稻谷合约",
"MA9999.XZCE":"甲醇合约",
"LR9999.XZCE":"晚籼稻合约",
"SF9999.XZCE":"硅铁合约",
"SM9999.XZCE":"锰硅合约",
"ZC9999.XZCE":"动力煤合约",
"CY9999.XZCE":"棉纱合约",
"AP9999.XZCE":"苹果合约",
"CJ9999.XZCE":"红枣合约",
"UR9999.XZCE":"尿素合约",
"SA9999.XZCE":"纯碱合约",
"PF9999.XZCE":"短纤合约",
"PK9999.XZCE":"花生合约",
"SH9999.XZCE":"烧碱合约",
"PX9999.XZCE":"对二甲苯合约"
}

# 可供搜索的参数
1.k 用当前时间往前k天的数据进行相关性和协整检验的出期货对

2.open_value 价差大于均值加open_value倍标准差或价差小于均值减open_value倍标准差开仓

3.end_value 价差大于均值减end_value倍标准差 小于均值加end_value倍标准差平仓

4.每隔多少天检测一次

5.回归速率应该大于多少

6.筛选期货对的时候设定的相关系数阈值

# 对每一对期货对 因为价格连续不跳跃 开仓后必须平仓后才能再开仓

用log(价格）进行协整检验

In [24]:
import math
import datetime
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

from rich.progress import track
from tqdm import trange

#生成所有的期货对 和带带信号阈值期货对
pair_list=[]
pair_list_open_end_value=[]
for i3 in Close.columns:
    for j3 in Close.columns:
        if i3!=j3 and set([i3,j3]) not in pair_list:
            pair_list.append(set([i3,j3]))
            pair_list_open_end_value.append(set([i3,j3]))

#给每个期货对弄个开仓与否的信号标识符
for f in range(len(pair_list)):
    pair_list[f]=[pair_list[f],0]
    pair_list_open_end_value[f]=[pair_list_open_end_value[f],[0,0,0,0]]



#新建账户
cyf=account("dont_care","dont_care",10000000,Close)
sta_pair=[]

k=22#往前多少天的数据
trigger_day=22#多少天进行一次选期货对和阈值
open_value=1.3
end_value=1

value=pd.DataFrame(index=Close.index)
value["净值"]=1
#print(len(Close.index[555*100:len(Close.index)]))

all_limit=pd.DataFrame(index=Close.index)

for d_index in trange(len(Close.index[555*k:len(Close.index)-555*k])):
   
    d=Close.index[555*k:len(Close.index)-555*k][d_index]
    current_index=np.where(Close.index==d)[0][0]

    if current_index%(555*trigger_day)==0:#每10天算下相关系数，协整检验
        
        tem_close_all=Close.iloc[current_index-1-555*k:current_index-1,]#当前时间之前的往前90天的所有数据
        corr = tem_close_all.corr('spearman')#相关系数
    
        #找到相关系数比较大的期货对
        alist=[]
        for i0 in corr.index:
            for j0 in corr.columns:
                if corr.loc[i0,j0]>0.9 and i0!=j0 and set([i0,j0]) not in alist:
                    alist.append(set([i0,j0]))
    
        #储存这些期货对以及它们的相关系数
        blist=[]
        for item in alist:
            fu_fu=list(item)
            blist.append([fu_fu,corr.loc[fu_fu[0],fu_fu[1]]])


        #进行协整检验            
        sta_pair=[]
        for i1 in range(len(blist)):

            tem_close=tem_close_all[[blist[i1][0][0],blist[i1][0][1]]].dropna()
            first_pair=np.log(tem_close[blist[i1][0][0]])
            second_pair=np.log(tem_close[blist[i1][0][1]])
            beta, c, resid, result=cointegration_check(first_pair,second_pair)

            if result==1.0:
                sta_pair.append([blist[i1][0][0],blist[i1][0][1],[0,0]])#后面这两个0存数量吗["a","b",[0,0]]
        print("符合条件的期货对的数量",len(sta_pair))

        if len(sta_pair)!=0:
        
            tem_close=Close.iloc[current_index-1-555*k:current_index-1,]
            #对于通过协整检验的期货对，计算出价差
            
            for i in range(len(sta_pair)):

                diff_price=pd.DataFrame(abs(tem_close[sta_pair[i][1]]-tem_close[sta_pair[i][0]]))
                diff_price.columns=["价差"]
                
                   
                    
                   
                    
                if diff_price.dropna().index[0]<\
                str(datetime.datetime.strptime(d,"%Y-%m-%d %H:%M:%S")- datetime.timedelta(days=k)):#起码要有k个交易日以上数据
                    
                    OU_params_hat = estimate_OU_params(np.array(diff_price["价差"].dropna()))
                    if OU_params_hat.alpha>0:
                        print(sta_pair[i][0],sta_pair[i][1],"alpha:",OU_params_hat.alpha,\
                              "相关系数:",corr.loc[sta_pair[i][0],sta_pair[i][1]])
                        
                        #mean_diff=np.mean(np.array(diff_price["价差"].dropna()))
                        #std_diff=math.sqrt(np.var(np.array(diff_price["价差"].dropna()))/(2*OU_params_hat.alpha))#这个alpha应该大于0
                        mean_diff=OU_params_hat.gamma
                        std_diff=OU_params_hat.beta
                        c1=mean_diff+open_value*std_diff
                        c2=mean_diff+end_value*std_diff
                        c3=mean_diff-end_value*std_diff
                        c4=mean_diff-open_value*std_diff
                        #把上一个周期的阈值重置为0
                        for pair in pair_list_open_end_value:
                            pair[1]=[0,0,0,0] 
                        #把这个周期的阈值存下来
                        for pair in pair_list_open_end_value:
                            if sta_pair[i][0] in pair[0] and sta_pair[i][1] in pair[0]:#["a","b",[0,0]]
                                pair[1]=[c1,c2,c3,c4]
                        try:        
                            print("接下来的图是"+sta_pair[i][0]+"_"+futures_name[sta_pair[i][0]]+"和"\
                                  +sta_pair[i][1]+"_"+futures_name[sta_pair[i][1]]+"的价差")
                            
                            future_close=Close.iloc[current_index+10:current_index+10+555*trigger_day,]
                            #假设用10分钟算完期货对,并往后推trigger_day天
                            
                            future_diff_price=pd.DataFrame(abs(future_close[sta_pair[i][1]]-future_close[sta_pair[i][0]]))
                            future_diff_price.columns=["different_price"]
                            future_diff_price=future_diff_price.dropna()
                            future_diff_price["mean"]=mean_diff
                            future_diff_price["up_open"]=c1
                            future_diff_price["up_end"]=c2
                            future_diff_price["down_end"]=c3
                            future_diff_price["down_open"]=c4
                            #future_diff_price.plot(figsize=(15,6))    
                            #plt.show()
                            
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"different_price" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"different_price"]]=np.nan   
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"std_diff" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"std_diff"]]=np.nan
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"mean" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"mean"]]=np.nan
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_open" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_open"]]=np.nan
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_end" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_end"]]=np.nan
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_end" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_end"]]=np.nan
                            if sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_open" not in all_limit.columns:
                                all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_open"]]=np.nan

                            #使用一下注释掉的语句会出现值全部是nan
                            #all_limit.loc[:,[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"different_price"]]\
                            #=pd.DataFrame(abs(Close[sta_pair[i][1]]-Close[sta_pair[i][0]]))
                            
                            all_limit[[sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"different_price"]]\
                            =pd.DataFrame(abs(Close[sta_pair[i][1]]-Close[sta_pair[i][0]]))
                            
                            
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"mean"]]=mean_diff
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"std_diff"]]=std_diff
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_open"]]=c1
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"up_end"]]=c2
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_end"]]=c3
                            all_limit.loc[current_index+10:current_index+10+555*trigger_day,\
                                          [sta_pair[i][0]+"_"+sta_pair[i][1]+"_"+"down_open"]]=c4
                            all_limit.to_csv("all_limit.csv")
                            print("数据保存结束")
                        except:
                            print("数据已经结束了，没有  当前分钟往后10分钟  至  当前分钟往后10分钟加trigger_day天的数据")


  0%|                                                                             | 7/373576 [00:00<3:06:06, 33.45it/s]

符合条件的期货对的数量 0


  3%|██▍                                                                      | 12209/373576 [04:39<2:13:26, 45.13it/s]

符合条件的期货对的数量 21
AG9999.XSGE AU9999.XSGE alpha: 0.00014222829105659606 相关系数: 0.9624548877204162
接下来的图是AG9999.XSGE_白银合约和AU9999.XSGE_黄金合约的价差
数据保存结束
EG9999.XDCE B9999.XDCE alpha: 0.0022779668500572674 相关系数: 0.9083744428742222
接下来的图是EG9999.XDCE_乙二醇合约和B9999.XDCE_豆二合约的价差


  3%|██▍                                                                      | 12209/373576 [04:50<2:13:26, 45.13it/s]

...

数据保存结束
C9999.XDCE CS9999.XDCE alpha: 0.0034648736507842453 相关系数: 0.9318052451098005
接下来的图是C9999.XDCE_玉米合约和CS9999.XDCE_玉米淀粉合约的价差
数据保存结束
CF9999.XZCE CY9999.XZCE alpha: 0.005844046814724785 相关系数: 0.9751891042481632
接下来的图是CF9999.XZCE_棉花合约和CY9999.XZCE_棉纱合约的价差
数据保存结束
L9999.XDCE PP9999.XDCE alpha: 0.005509243738771685 相关系数: 0.9558404695621346
接下来的图是L9999.XDCE_聚乙烯合约和PP9999.XDCE_聚丙烯合约的价差
数据保存结束
V9999.XDCE L9999.XDCE alpha: 0.008979842259597874 相关系数: 0.9431022788545285
接下来的图是V9999.XDCE_聚氯乙烯合约和L9999.XDCE_聚乙烯合约的价差
数据保存结束
M9999.XDCE RM9999.XZCE alpha: 0.010227832133735498 相关系数: 0.9879262356802289
接下来的图是M9999.XDCE_豆粕合约和RM9999.XZCE_菜籽粕合约的价差
数据保存结束
NR9999.XINE RU9999.XSGE alpha: 0.00573830767736894 相关系数: 0.9436037175659423
接下来的图是NR9999.XINE_20号胶合约和RU9999.XSGE_天然橡胶合约的价差
数据保存结束
Y9999.XDCE P9999.XDCE alpha: 0.0008531352405324339 相关系数: 0.9819229818617091
接下来的图是Y9999.XDCE_豆油合约和P9999.XDCE_棕榈油合约的价差
数据保存结束
SF9999.XZCE SM9999.XZCE alpha: 0.0032951261806046486 相关系数: 0.9576840669843308
接下来的图是SF9999.XZCE_硅铁合约和SM9

  7%|████▌                                                                 | 24426/373576 [17:39<1032:00:53, 10.64s/it]

数据保存结束


  9%|██████▋                                                                  | 34478/373576 [21:26<3:30:50, 26.80it/s]


KeyboardInterrupt: 

In [ ]:
    else:#每10天除了算期货对和阈值的那天
        
        if len(sta_pair)!=0:
           
            #交易
            one_pair_coin=get_total_value_of_futures_in_a_moment(cyf,d,Close)/len(sta_pair)#把资金均分到每一个期货对上
            if str(one_pair_coin)!="nan":#比如黄金在夜晚交易时间，它对应的比如大豆不在交易时间就不行
                for i in range(len(sta_pair)):
                    #对于每一对期货对 
                
                    flag=pair_list[find_pair_index(pair_list,sta_pair[i][0],sta_pair[i][1])][1]
                    #关于这个运行逻辑是这样的，一开始期货对的数量为0，阈值也是没有的。。。然后第一次生成一些期货对，阈值进行更新。。
                    #然后第二次生成一些期货对，（此时第二次生成的期货对与第一次生成的期货对这个集合不相等），在第一次期货对不在第二次期货对
                    #的一些元素，阈值不为0，但是sta_pair更新了，无所谓
                    #不会出现在第一次期货对不在第二次期货对但是进行了交易的情况
                
                    for pair in pair_list_open_end_value:
                        if sta_pair[i][0] in pair[0] and sta_pair[i][1] in pair[0] and pair[1]!=[0,0,0,0]:
 
                            c1=pair[1][0]
                            c2=pair[1][1]
                            c3=pair[1][2]
                            c4=pair[1][3]
            
            
                            price1=get_price(sta_pair[i][0],d,Close)
                            price2=get_price(sta_pair[i][1],d,Close)
                            half_price=one_pair_coin/2
    
                            if str(price1)!="nan" and str(price2)!="nan":
                                amount1=int(half_price/price1)#0
                                amount2=int(half_price/price2)#1
                        
                                #开仓
                                if abs(Close.at[d,sta_pair[i][1]]-Close.at[d,sta_pair[i][0]])>c1 \
                                and flag==0:
                                    pair_list[find_pair_index(pair_list,sta_pair[i][0],sta_pair[i][1])][1]=1#把期货对的状态记录改为开仓
                                    print("价差过大开仓",d,sta_pair[i][0],sta_pair[i][1])
                                    if Close.at[d,sta_pair[i][1]]>=Close.at[d,sta_pair[i][0]]:#1价格高 0价格低
                    
                                        order_sell_futures(cyf,sta_pair[i][1],d,amount2,0.05,Close,Volume)
                                        order_buy_futures(cyf,sta_pair[i][0],d,amount1,0.05,Close,Volume)
                                        sta_pair[i][2]=[amount1,-amount2]
                                    else:
                                        order_sell_futures(cyf,sta_pair[i][0],d,amount1,0.05,Close,Volume)
                                        order_buy_futures(cyf,sta_pair[i][1],d,amount2,0.05,Close,Volume)
                                        sta_pair[i][2]=[-amount1,amount2]
            
                                elif abs(Close.at[d,sta_pair[i][1]]-Close.at[d,sta_pair[i][0]])<c2 \
                                and flag==0:
                                    pair_list[find_pair_index(pair_list,sta_pair[i][0],sta_pair[i][1])][1]=1#把期货对的状态记录改为开仓
                                    print("价差过小开仓",d,sta_pair[i][0],sta_pair[i][1])
                                    if Close.at[d,sta_pair[i][1]]>=Close.at[d,sta_pair[i][0]]:#1价格高 0价格低
                                        order_sell_futures(cyf,sta_pair[i][0],d,amount1,0.05,Close,Volume)
                                        order_buy_futures(cyf,sta_pair[i][1],d,amount2,0.05,Close,Volume)
                                        sta_pair[i][2]=[-amount1,amount2]
                                    else:
                                        order_sell_futures(cyf,sta_pair[i][1],d,amount2,0.05,Close,Volume)
                                        order_buy_futures(cyf,sta_pair[i][0],d,amount1,0.05,Close,Volume)
                                        sta_pair[i][2]=[amount1,-amount2]
            
                                #平仓 这个平仓有问题 重新计算期货对的时候把平仓给重置了
                                elif c3<abs(Close.at[d,sta_pair[i][1]]-Close.at[d,sta_pair[i][0]])<c4:
                                    print("平仓",d,sta_pair[i][0],sta_pair[i][1])
                                    pair_list[find_pair_index(pair_list,sta_pair[i][0],sta_pair[i][1])][1]=0#把期货对的状态记录改为平仓
                                    print("平仓",d)
                                    end_transation(account,sta_pair[i],d,Close)

            

    value.loc[d,["净值"]]=get_total_value_of_futures_in_a_moment(cyf,d,Close)/10000000

    #最后一天统统平仓
for i2 in range(len(Close.columns)): 
    tem_close=Close.iloc[len(Close.index)-555*30:len(Close.index),i2].dropna()
    

    end_transation_one_futures(cyf,Close.columns[i2],cyf.stock[Close.columns[i2]],tem_close.index[-1],Close)
cyf.get_coin()

In [ ]:
unique_name=[]
for name in all_limit.columns:
    name_list=name.split("_")
    uq_name=name_list[0]+"_"+name_list[1]+"_"
    if uq_name not in unique_name:
        unique_name.append(uq_name)

for uq in unique_name:        
    dmuudd=[uq+ x for x in ["different_price","mean","up_open","up_end","down_end","down_open"]]
    all_limit[dmuudd].dropna().plot()
    plt.show()

In [ ]:
value["1"]=1
value.plot(figsize=(18,6))

In [ ]:
l=90
for i in range(len(sta_pair)):
    diff_price=pd.DataFrame(abs(Close[sta_pair[i][1]]-Close[sta_pair[i][0]]))
    diff_price.columns=[sta_pair[i][1]+" and "+sta_pair[i][0]+"价差"]
    diff_price["mean"]=np.nan
    diff_price["up_open"]=np.nan
    diff_price["down_open"]=np.nan
    for d in range(555*30,len(Close.index)):
        if d%555*10==0:

            mean_diff,up_open,down_open,up_end,down_end=generate_up_down(d,l,i,k)
            try:
                diff_price.loc[Close.index[d]:Close.index[d+555*10],"mean"]=mean_diff
                diff_price.loc[Close.index[d]:Close.index[d+555*10],"up_open"]=up_open
                diff_price.loc[Close.index[d]:Close.index[d+555*10],"down_open"]=down_open
                diff_price.loc[Close.index[d]:Close.index[d+555*10],"up_end"]=up_end
                diff_price.loc[Close.index[d]:Close.index[d+555*10],"down_end"]=down_end
            except:#收尾处
                continue
    
    if diff_price.dropna().index[0]<"2023-03-01":
        diff_price.dropna().plot(figsize=(12,6))

# 搜索参数？？？

In [ ]:
mport numpy as np
from pymoo.core.problem import ElementwiseProblem

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


problem = MyProblem()

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=40,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 40)

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

In [ ]:
import matplotlib.pyplot as plt
xl, xu = problem.bounds()
plt.figure(figsize=(7, 5))
plt.scatter(X[:, 0], X[:, 1], s=30, facecolors='none', edgecolors='r')
plt.xlim(xl[0], xu[0])
plt.ylim(xl[1], xu[1])
plt.title("Design Space")
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()

# 参考

https://www.zhihu.com/question/268075949

https://zhuanlan.zhihu.com/p/28736174